## Pipeline de Modelado IA para Recomendación de Locales

Este notebook tiene como objetivo entrenar y evaluar un modelo de aprendizaje automático para recomendar las mejores zonas para abrir un nuevo negocio. Se basa en los datos enriquecidos con la API de Google Places y generados en el notebook anterior.

⚠️ **Importante:** Este dataset es una **muestra representativa** (20% por código postal) del total de datos para agilizar las pruebas y el desarrollo.

### ¿Qué incluye este notebook?
- **Carga del dataset enriquecido** con variables como afluencia, competencia y valoraciones.
- **Selección de características** relevantes para el modelo.
- **Entrenamiento** de un modelo de Random Forest para predecir la valoración esperada de un local.
- **Evaluación del modelo** con métricas de desempeño (RMSE y R²).
- **Exportación del modelo entrenado** para su uso posterior en la generación de informes de recomendación.
- **Exportacion de visualizaciones del modelo**

In [1]:
import pandas as pd
import numpy as np
import pickle
import os
import matplotlib.pyplot as plt
import seaborn as sns
import shutil

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Carpetas de salida
IMAGES_DIR = "../images"
ASSETS_IMAGES_DIR = "../../streamlit/assets/images"
DATA_DIR = "../data"
ASSETS_DATA_DIR = "../../streamlit/assets/data"
os.makedirs(IMAGES_DIR, exist_ok=True)
os.makedirs(DATA_DIR, exist_ok=True)

# Cargar el CSV enriquecido
df = pd.read_csv('../data/locales_enriquecido_muestra.csv')

print(f"Dataset cargado con {df.shape[0]} filas y {df.shape[1]} columnas.")

Dataset cargado con 579 filas y 28 columnas.


In [2]:
# Features: todas las variables numéricas y relevantes para el modelo
features = [
    'puntuacion_media', 'numero_reviews', 'categoria_id',
    'density_500m', 'density_1000m', 'density_2000m',
    'ratio_500m_2km', 'dist_city_center_km', 'local_density_1km',
    'dist_city_center_km^2', 'density_1000m^2',
    'valoracion_norm', 'valoracion_por_cercania',
    'competencia_count', 'competencia_rating'
]

# Target: ajusta según lo que quieras predecir (por ejemplo, 'valoracion')
target = 'valoracion'

In [3]:
X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Train: {X_train.shape}, Test: {X_test.shape}")

Train: (463, 15), Test: (116, 15)


In [4]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predicciones
y_pred = model.predict(X_test)

In [5]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.4f}")
print(f"R^2: {r2:.4f}")

RMSE: 0.1516
R^2: 0.9730


In [6]:
# Guardar el modelo entrenado
model_path = '../models/random_forest_model.pkl'
with open(model_path, 'wb') as file:
    pickle.dump(model, file)


# Rutas de exportación
model_path_1 = os.path.join("..", "models", "random_forest_model.pkl")
model_path_2 = os.path.join("..", "..", "streamlit", "assets", "models", "random_forest_model.pkl")

# Guardar el modelo en ambas rutas
for path in [model_path_1, model_path_2]:
    with open(path, 'wb') as file:
        pickle.dump(model, file)
    print(f"Modelo exportado correctamente en: {path}")


Modelo exportado correctamente en: ../models/random_forest_model.pkl
Modelo exportado correctamente en: ../../streamlit/assets/models/random_forest_model.pkl


### VISUALIZACIONES Y RESULTADOS

In [7]:
## 1. Importancia de Variables
output_filename = "importancia_variables.png"
output_path_1 = os.path.join(IMAGES_DIR, output_filename)

plt.figure(figsize=(10,6))
plt.barh(X_train.columns, model.feature_importances_)
plt.xlabel("Importancia")
plt.ylabel("Características")
plt.title("Importancia de Variables")
plt.tight_layout()
plt.savefig(output_path_1)
plt.close()

# Copiar al assets
output_path_2 = os.path.join(ASSETS_IMAGES_DIR, output_filename)
shutil.copy(output_path_1, output_path_2)
print(f"Importancia de Variables exportada en:\n- {output_path_1}\n- {output_path_2}")

## 2. Real vs Predicho
output_filename = "real_vs_predicho.png"
output_path_1 = os.path.join(IMAGES_DIR, output_filename)

y_pred = model.predict(X_test)
plt.figure(figsize=(8,6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.xlabel("Valor real")
plt.ylabel("Valor predicho")
plt.title("Real vs Predicho")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.tight_layout()
plt.savefig(output_path_1)
plt.close()

output_path_2 = os.path.join(ASSETS_IMAGES_DIR, output_filename)
shutil.copy(output_path_1, output_path_2)
print(f"Real vs Predicho exportado en:\n- {output_path_1}\n- {output_path_2}")

## 3. Histograma de Errores Absolutos
output_filename = "histograma_errores.png"
output_path_1 = os.path.join(IMAGES_DIR, output_filename)

errors = np.abs(y_test - y_pred)
plt.figure(figsize=(8,5))
plt.hist(errors, bins=30, color='skyblue', edgecolor='black')
plt.xlabel("Error Absoluto")
plt.ylabel("Frecuencia")
plt.title("Histograma de Errores Absolutos")
plt.tight_layout()
plt.savefig(output_path_1)
plt.close()

output_path_2 = os.path.join(ASSETS_IMAGES_DIR, output_filename)
shutil.copy(output_path_1, output_path_2)
print(f"Histograma de Errores exportado en:\n- {output_path_1}\n- {output_path_2}")


Importancia de Variables exportada en:
- ../images/importancia_variables.png
- ../../streamlit/assets/images/importancia_variables.png
Real vs Predicho exportado en:
- ../images/real_vs_predicho.png
- ../../streamlit/assets/images/real_vs_predicho.png
Histograma de Errores exportado en:
- ../images/histograma_errores.png
- ../../streamlit/assets/images/histograma_errores.png


In [8]:
# Cálculo de métricas
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

metricas = {
    "RMSE": [rmse],
    "R2": [r2],
    "MAE": [mae]
}
metricas_df = pd.DataFrame(metricas)

# Guardar el archivo principal
output_filename = "metricas_regresion.csv"
output_path_1 = os.path.join(DATA_DIR, output_filename)
metricas_df.to_csv(output_path_1, index=False)

# Copiar al assets
output_path_2 = os.path.join(ASSETS_DATA_DIR, output_filename)
shutil.copy(output_path_1, output_path_2)

print(f"Métricas exportadas en:\n- {output_path_1}\n- {output_path_2}")


Métricas exportadas en:
- ../data/metricas_regresion.csv
- ../../streamlit/assets/data/metricas_regresion.csv
